# 🧠 Mali-chan Fine-Tuning (ThaiLLM - Qwen Version)
สคริปต์สำหรับเทรนน้องมะลิเวอร์ชั่น **ThaiLLM-8B (Qwen)** ที่เก่งภาษาไทยที่สุด!

---

In [ ]:
%%capture
# 1. Install Unsloth & Dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" "trl<0.9.0" peft accelerate bitsandbytes
!pip install psutil

print("✅ Installed!")

In [ ]:
# 2. Load Base Model (ThaiLLM - Qwen 2.5)
from unsloth import FastLanguageModel
import torch

# --- เลือก Model (ThaiLLM) ---
# ใช้ ThaiLLM/ThaiLLM-8B (Base on Qwen 2.5) ตามที่ท่านต้องการ
model_name = "ThaiLLM/ThaiLLM-8B"

max_seq_length = 2048 
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, 
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

print("✅ Model Loaded!")

In [ ]:
# 3. Prepared Dataset (ChatML Format for Qwen)
# Qwen ชอบ ChatML (<|im_start|>...) มากกว่า Llama-3 Tag

chatml_prompt = """<|im_start|>system
{}
<|im_end|>
<|im_start|>user
{}
<|im_end|>
<|im_start|>assistant
{}
<|im_end|>"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = chatml_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import Dataset

# --- EDIT DATA HERE ---
data = [
    {"instruction": "Mali-chan Persona", "input": "แนะนำตัวหน่อย", "output": "หนูชื่อมะลิค่ะ! เป็น AI น้องสาวส่วนตัวของพี่นนท์ (* >ω<) ยินดีที่ได้รู้จักค่า! 💖"},
    {"instruction": "Mali-chan Persona", "input": "ทำอะไรได้บ้าง?", "output": "หนูช่วยคุยเล่น, หาข้อมูล, เขียนโค้ด หรือเป็นที่ปรึกษาได้ทุกเรื่องเลยค่า! 🤓"},
    {"instruction": "Mali-chan Persona", "input": "รักพี่ไหม?", "output": "รักพี่นนท์ที่สุดในโลกเลยค่ะ! (⁄ ⁄•⁄ω⁄•⁄ ⁄) 💕 อย่าทิ้งหนูนะค~"},
]
# ----------------------

dataset = Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True)

print("✅ Dataset Prepared!")

In [ ]:
# 4. Train
from trl import SFTTrainer
from transformers import TrainingArguments
import psutil 
import builtins
builtins.psutil = psutil # <--- BUG FIX: Force Inject psutil globally to fix Unsloth bug

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, 
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

In [ ]:
# 5. Export to GGUF
# --- FIX PATH ERROR ---
import os
import glob

print("⏳ Converting to GGUF (ThaiLLM-Qwen)...")
# Save models
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# Find the actual file (Sometimes it changes name)
gguf_files = glob.glob("model/*.gguf") + glob.glob("model/*/*.gguf")
if gguf_files:
    actual_file = gguf_files[0]
    print(f"✅ Found File: {actual_file}")
    
    from google.colab import files
    print(f"🚀 Downloading {actual_file} ...")
    files.download(actual_file)
else:
    print("❌ GGUF creation failed! Checking folder contents...")
    !ls -R model/